In [3]:
# Just to define file paths, not related to sisl
from pathlib import Path

# Sisl imports
import sisl
import sisl.viz
import numpy as np
import scipy

# To quickly plot the hamiltonian matrix
import plotly.express as px

import numpy as np
import torch
import yaml
import importlib
import random

# So that we can plot sisl geometries
import sisl.viz

from e3nn import o3
from pathlib import Path

from graph2mat import (
    PointBasis,
    BasisTableWithEdges,
    BasisConfiguration,
    MatrixDataProcessor,
)
from graph2mat.bindings.torch import TorchBasisMatrixData, TorchBasisMatrixDataset
from graph2mat.bindings.e3nn import E3nnGraph2Mat

def load_config(path="../config.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)
    
def flatten(xss):
    return [x for xs in xss for x in xs]

#Get the fdf file for the calculation
path_uc = Path("../dataset/SHARE_OUTPUTS_8_ATOMS/0a2a-fbca-4649-8012-f5aa640bfd1d")   #SIESTA calculation folder
fdf_uc = sisl.get_sile(path_uc / "aiida.HSX")
geometry = fdf_uc.read_geometry()
H_uc = fdf_uc.read_hamiltonian()
cell=geometry.cell

def reduced_coord(kpt, cell):
    """Convert a k-point to reduced coordinates."""
    return (cell.T)@kpt/(2*np.pi)

In [ ]:
# We will rely for now on the shifts generated by siesta. Because their computation would require a lot of effort. However, we should be able to compute them by just knowing the atom orbital positions and types, which we presumably know.

# # TODO: Adapt this to sparse format for matrix_uc
# def reconstruct_tim(matrix_uc, k_point, cell, isc_list, isc_idx_list):
#     n_rows = matrix_uc.shape[0] 
#     n_cols = matrix_uc.shape[1]  
#     n_shifts = n_cols // n_rows  

#     # Get the indices of the shifts

#     tim_manual = np.zeros((n_rows, n_rows), dtype=np.complex128)
#     blocks = []
#     for isc_idx in isc_idx_list:
#         # Compute the actual shift, not only the "integer jumps"
#         isc = isc_list[isc_idx]
#         shift = np.array(np.sum([cell[i] * isc[i] for i in range(len(isc))], axis=0))

#         block = matrix_uc[0:n_rows, isc_idx:isc_idx+n_rows] * np.exp(-1j*np.dot(k_point, shift))
#         tim_manual +=  block # sum all n_rows x n_rows blocks of H_matrix (at k=0, all phase factors are 1)
#         block[block == 0] = None  # replace zeros with None for better visualization
#         blocks.append(block)
    
#     return tim_manual

def ReconstructH(k_point, H_coo, orb_i, orb_j, isc, cell):

    no=H_coo.shape[0]    #number of rows of H_matrix, i.e. number of orbitals
    H_g=np.zeros((no, no), dtype=complex) #no x no matrix of zeros for our H(k) which we wil now fill (it may be wasteful to use a dense matrix for that, but we do it here for simplicity)

    for k in range(0, H_coo.nnz):
        Ruc = np.array(np.sum([cell[i] * isc[k, i] for i in range(len(isc))], axis=0))  #unit cell to unit cell repetition distance (i.e. cell phase convention)
        Phase=np.exp(1j*k_point.dot(Ruc))
        H_g[orb_i[k],orb_j[k]]+=H_coo.data[k]*Phase   #add contribution to correct entry of H(k)

    return H_g

H_coo = H_uc.tocsr().tocoo()  # Convert the Hamiltonian to COO format
S_uc = fdf_uc.read_overlap() 
S_coo = S_uc.tocsr().tocoo()  # Convert the Overlap to COO format

# Orbital indices and shifts extraction
orb_i=-np.ones(H_coo.nnz, dtype=int) #we fill everything with minus ones to be able to detect iif something went wrong
orb_j=-np.ones(H_coo.nnz, dtype=int) #same
isc=-np.ones((H_coo.nnz,3), dtype=int) #same

orb_i=H_coo.row
for k in range(H_coo.nnz):
    orb_j[k]=geometry.osc2uc(H_coo.col[k])   #unit cell orbital indices corresponding to the "supercell" indices
    isc[k]=geometry.o2isc(H_coo.col[k])      #which unit cell / shift that orbital belongs to wrt. the (0,0,0) unit cell

# K path
kzs=np.linspace(0,1, num=50)
kdir=geometry.rcell[:,2]
k_path=np.array([kz*kdir for kz in kzs])

# TIM reconstruction
energy_bands = []
for k_point in k_path:
    Hk = ReconstructH(k_point, H_coo, orb_i, orb_j, isc, cell)
    Sk = ReconstructH(k_point, S_coo, orb_i, orb_j, isc, cell)

    Ek = scipy.linalg.eigh(Hk, Sk, eigvals_only=True)
    print(Ek.shape)
    a

    energy_bands.append(Ek)



    

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
# === List of paths to all structures ===
parent_path = Path('../dataset')
n_atoms_paths = list(parent_path.glob('*/'))
paths = []
for n_atoms_path in n_atoms_paths:
    structure_paths = list(n_atoms_path.glob('*/'))
    paths.append(structure_paths)
paths = flatten(paths)

random.seed(42)
random.shuffle(paths)

# Test TIM reconstruction in sparse format

In [ ]:
k_point = np.array([3,0,5])
# atol=None
# rtol=None

tim_sisl = H_uc.Hk(reduced_coord(k_point, cell), gauge='cell').toarray()
# tim_manual = reconstruct_tim(H_uc.tocsr().todense(), k_point, cell, isc_list, isc_idx_list)
tim_manual, _ = ReconstructH(k_point, H_coo, S_coo, orb_i, orb_j, isc, cell)

tim_sisl_plot = np.abs(tim_sisl)
tim_manual_plot = np.abs(tim_manual)

are_close = np.allclose(tim_sisl, tim_manual)

print(are_close)
px.imshow(tim_sisl_plot, title="SISL TIM at Gamma").show()
px.imshow(tim_manual_plot, title="Manual TIM at Gamma").show()

True


# Silly tests

In [32]:
from scipy.sparse import coo_matrix

matrix = np.array([[1,0,2], [0,1,0], [3,0,5]])
matrix = coo_matrix(matrix)
print(matrix.col)

[0 2 1 0 2]
